### **Agenda**

1. Built in functions in Runnables 
2. Combining Chains 
3. RAG Demo - LCEL 

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI 

model = ChatOpenAI(model = "gpt-3.5-turbo-0125")

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate 

In [6]:
prompt = ChatPromptTemplate.from_template("tell me a curios fact about {soccer_player}")

output_parser = StrOutputParser()

In [8]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he is the first player in history to score more than 700 career goals.'

**use of .bind() to add arguments to a Runnable in a LCEL Chain**

- For example, we can add an argument to stop the model response when it reaches the word "Ronaldo"

In [9]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

'One interesting fact about Cristiano '

**Combining LCEL Chain (Chain inside chain)**

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate 

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()

In [13]:
chain.invoke("Chamberlain")

'Chamberlain was a British politician known for his policy of appeasement towards Nazi Germany in the lead up to World War II.'

In [14]:
historian_prompt = ChatPromptTemplate.from_template("was {politician} positive for Humanity")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [15]:
composed_chain.invoke({"politician": "Lincoln"})

'Yes, Abraham Lincoln is viewed positively for his leadership during a critical time in American history. His efforts to preserve the Union and end slavery have had a lasting impact on the country and are seen as important steps towards equality and justice for all. Many consider him to be one of the greatest presidents in American history for his dedication to upholding the principles of freedom and equality.'

**Another Example**

In [19]:
from operator import itemgetter 

from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
    )

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model 
    | StrOutputParser()
)

chain2.invoke({"politician": "Imran Khan Niazi", "language": "English"})



'Imran Khan Niazi is from Pakistan, which is located in the continent of Asia.'

### **LCEL chain at work in a typical RAG app**

In [20]:
from langchain_openai import ChatOpenAI 

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [26]:
import bs4
from langchain import hub
from langchain_chroma import Chroma 
from langchain_community.document_loaders import WebBaseLoader 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter 

loader = WebBaseLoader(
    web_path = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriver = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs (docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()}
    |prompt 
    |model 
    | StrOutputParser()
)





c:\Users\HP\miniconda3\envs\LC_series\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [27]:
rag_chain.invoke("what is Task Decomposition?")

'Task Decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable. Techniques like Chain of Thought and Tree of Thoughts help in decomposing tasks and exploring multiple reasoning possibilities at each step. Task decomposition can be achieved through various methods, including using prompting techniques, task-specific instructions, or human inputs.'